In [1]:
def get_opt_level(filename: str):
    """Extracts the optimization level based on a filename.

    Keyword arguments:
    filename -- filename of a benchmark

    Return values:
    num -- optimization level
    """

    pat = re.compile(r"opt\d")
    m = pat.search(filename)
    if m:
        num = m.group()[-1:]
    else:
        num = -1
    return int(num)

In [2]:
def get_num_qubits(filename: str):
    """Extracts the number of qubits based on a filename.

    Keyword arguments:
    filename -- filename of a benchmark

    Return values:
    num -- optimization level
    """

    pat = re.compile(r"(\d+)\.")
    m = pat.search(filename)
    if m:
        num = m.group()[0:-1]
    else:
        num = -1
    return int(num)

In [3]:
def get_tket_settings(filename: str):
    if "line" in filename:
        return "line"
    elif "graph" in filename:
        return "graph"

def get_gate_set(filename: str):
    if "oqc" in filename:
        return "oqc"
    elif "ionq" in filename:
        return "ionq"
    elif "ibm" in filename:
        return "ibm"
    elif "oqc" in filename:
        return "oqc"
    
    
def get_target_device(filename: str):
    if "ibm_washington" in filename:
        return "ibm_washington"
    elif "ibm_montreal" in filename:
        return "ibm_montreal"
    elif "rigetti_aspen_m1" in filename:
        return "rigetti_aspen_m1"
    elif "ionq11" in filename:
        return "ionq11"
    elif "oqc_lucy" in filename:
        return "oqc_lucy"

In [4]:
def get_compiler_and_settings(filename:str):
    if "qiskit" in filename:
        return ("qiskit", get_opt_level(filename))
    elif "tket" in filename:
        return ("tket", get_tket_settings(filename))

In [15]:
def parse_data(filename: str):
    """Extracts the necessary information from a given filename.

    Keyword arguments:
    filename -- name of file

    Return values:
    parsed_data -- parsed data extracted from filename
    """
    benchmark = filename.split("_")[1].lower().split("/")[-1]
    num_qubits = get_num_qubits(filename)
    indep_flag = "indep" in filename
    nativegates_flag = "nativegates" in filename
    mapped_flag = "mapped" in filename
    compiler, compiler_settings = get_compiler_and_settings(filename)
    gate_set = get_gate_set(filename)
    target_device = get_target_device(filename)

    path = os.path.join(filename)
    parsed_data = [
        benchmark,
        num_qubits,
        indep_flag,
        nativegates_flag,
        mapped_flag,
        compiler,
        compiler_settings,
        gate_set,
        target_device,
        path,
    ]
    return parsed_data

In [22]:
from mqt.bench.utils import utils, qiskit_helper, tket_helper
import os
import re
import pandas as pd
rows_list = []
# for filename in os.listdir(qasm_path):
directory = utils.get_qasm_output_path()
i= 0
for f in os.listdir(directory):
    i+=1
    parsed_data = parse_data(os.path.join(directory, f))
    rows_list.append(parsed_data)
    if i > 50:
        break

colnames = [
        "benchmark",
        "num_qubits",
        "indep_flag",
        "nativegates_flag",
        "mapped_flag",
        "compiler",
        "compiler_settings",
        "gate_set",
        "target_device",
        "path",
]

database = pd.DataFrame(rows_list, columns=colnames)
database["num_qubits"] = database["num_qubits"].astype(int)
database["benchmark"] = database["benchmark"].astype(str)

database.head(50)

,benchmark,num_qubits,indep_flag,nativegates_flag,mapped_flag,compiler,compiler_settings,gate_set,target_device,path
0,ae,27,False,True,False,qiskit,0,ionq,None,./benchviewer/static/files/qasm_output/ae_nati...
1,graphstate,24,False,True,False,qiskit,3,oqc,None,./benchviewer/static/files/qasm_output/graphst...
2,dj,41,False,False,True,qiskit,2,ibm,ibm_washington,./benchviewer/static/files/qasm_output/dj_mapp...
3,qft,8,True,False,False,tket,None,None,None,./benchviewer/static/files/qasm_output/qft_ind...
4,graphstate,19,False,True,False,qiskit,1,oqc,None,./benchviewer/static/files/qasm_output/graphst...
5,qaoa,3,False,False,True,tket,line,oqc,oqc_lucy,./benchviewer/static/files/qasm_output/qaoa_ma...
6,graphstate,4,False,True,False,qiskit,0,ibm,None,./benchviewer/static/files/qasm_output/graphst...
7,qftentangled,16,False,False,True,qiskit,2,None,rigetti_aspen_m1,./benchviewer/static/files/qasm_output/qftenta...
8,qpeinexact,25,False,True,False,qiskit,2,ibm,None,./benchviewer/static/files/qasm_output/qpeinex...
9,qpeinexact,18,False,True,False,qiskit,0,ibm,None,./benchviewer/static/files/qasm_output/qpeinex...


In [21]:
len(database)

11